In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from timeSeriesDataset import TimeSeriesDataset
from network import NeuralNetwork

In [2]:
batchSize = 512
inputLength = 900 # 3 hours
outputLength = 25 # 5 minutes

df = pd.read_csv('./allPrices.csv')
trainDataloader = DataLoader(TimeSeriesDataset(df[df['block']<20207949], 'block', 'price', 900, 25), batch_size=batchSize, shuffle=True)
testDataloader = DataLoader(TimeSeriesDataset(df[df['block']>=20207949], 'block', 'price', 900, 25), batch_size=batchSize)
# 20207949 on Ethereum occurred at Jul-01-2024 12:00:11 AM +UTC

for X, y in testDataloader:
    print(f"Shape of X [N, C, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, W]: torch.Size([512, 1, 900])
Shape of y: torch.Size([512, 25]) torch.float32


In [3]:
# preparing the model
device = 'cuda'
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Conv1d(1, 5, kernel_size=(100,), stride=(10,))
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): ReLU()
    (3): Linear(in_features=405, out_features=200, bias=True)
    (4): ReLU()
    (5): Linear(in_features=200, out_features=25, bias=True)
  )
)


In [4]:
lossFn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 1000 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

In [5]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainDataloader, model, lossFn, optimizer)
    test(testDataloader, model, lossFn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.233678  [  512/3940145]
loss: 0.174727  [512512/3940145]
loss: 0.155064  [1024512/3940145]
loss: 0.140673  [1536512/3940145]
loss: 0.109018  [2048512/3940145]
loss: 0.087594  [2560512/3940145]
loss: 0.086466  [3072512/3940145]
loss: 0.067633  [3584512/3940145]
Test Error: 
 Avg loss: 0.101228 

Epoch 2
-------------------------------
loss: 0.058548  [  512/3940145]
loss: 0.049477  [512512/3940145]
loss: 0.043652  [1024512/3940145]
loss: 0.040703  [1536512/3940145]
loss: 0.035716  [2048512/3940145]
loss: 0.030830  [2560512/3940145]
loss: 0.024599  [3072512/3940145]
loss: 0.029425  [3584512/3940145]
Test Error: 
 Avg loss: 0.038098 

Epoch 3
-------------------------------
loss: 0.027262  [  512/3940145]
loss: 0.028296  [512512/3940145]
loss: 0.026591  [1024512/3940145]
loss: 0.024777  [1536512/3940145]
loss: 0.025552  [2048512/3940145]
loss: 0.026065  [2560512/3940145]
loss: 0.026162  [3072512/3940145]
loss: 0.024646  [3584512/3940145]
Tes

In [6]:
torch.save(model.state_dict(), "model.pth")

In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [7]:
import matplotlib.pyplot as plt

In [19]:
model.eval()
testData = TimeSeriesDataset(df[df['block']>=20207949], 'block', 'price', 900, 25) # to avoid re-running
x, y = testData[0][0].unsqueeze(0), testData[0][1].unsqueeze(0)
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    print(pred, y)

tensor([[0.6602, 0.6604, 0.6710, 0.6883, 0.6643, 0.6634, 0.6855, 0.6697, 0.6892,
         0.6763, 0.6669, 0.7100, 0.6766, 0.6603, 0.6796, 0.6525, 0.6765, 0.6624,
         0.6685, 0.6847, 0.6721, 0.6580, 0.6888, 0.6556, 0.6523]],
       device='cuda:0') tensor([[0.7053, 0.7053, 0.7053, 0.7053, 0.7053, 0.7053, 0.7006, 0.7006, 0.7006,
         0.7006, 0.7006, 0.7006, 0.7010, 0.7010, 0.7010, 0.7014, 0.7014, 0.7014,
         0.7014, 0.7010, 0.7007, 0.7007, 0.7007, 0.7007, 0.7007]])
